In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from openpyxl.drawing.image import Image
from io import BytesIO
from math import sin, cos, exp, sqrt, pi
from typing import Callable, Tuple, List, Dict

In [3]:
# Evaluation functions
def f1(x: np.ndarray) -> float:
    """
    Example evaluation function f1.

    Args:
        x (np.ndarray): Input vector.

    Returns:
        float: Evaluated fitness value.
    """
    squared_sum = np.sum(x ** 2)
    return -5 / (1 + squared_sum) + sin(1 / np.tan(exp(-5 / (1 + squared_sum))))

def f2(x: np.ndarray, a: float = 20, b: float = 0.2, c: float = 2 * pi) -> float:
    """
    Ackley function, commonly used as a benchmark in optimization problems.

    Args:
        x (np.ndarray): Input vector.
        a (float): Parameter a, default is 20.
        b (float): Parameter b, default is 0.2.
        c (float): Parameter c, default is 2 * pi.

    Returns:
        float: Evaluated fitness value.
    """
    n = len(x)
    sum_sq_term = -a * exp(-b * sqrt(np.sum(x ** 2) / n))
    cos_term = -exp(np.sum(np.cos(c * x)) / n)
    return sum_sq_term + cos_term + a + exp(1)

In [4]:
def binary_to_real(binary_population: np.ndarray, domain: Tuple[float, float], dimensions: int) -> np.ndarray:
    """
    Convert a binary population to a real-valued matrix.

    Args:
        binary_population (np.ndarray): Binary encoded population.
        domain (Tuple[float, float]): Lower and upper bounds for real values.
        dimensions (int): Number of dimensions.

    Returns:
        np.ndarray: Real-valued representation of the population.
    """
    max_value = 2 ** 16 - 1
    binary_values = binary_population.reshape(-1, dimensions, 16)
    int_values = binary_values.dot(2 ** np.arange(15, -1, -1))
    real_values = domain[0] + (int_values / max_value) * (domain[1] - domain[0])
    return real_values

In [6]:
def initialize_population(pop_size: int, dimensions: int, domain: Tuple[float, float], binary: bool = False) -> np.ndarray:
    """
    Initialize a population of individuals.

    Args:
        pop_size (int): Population size.
        dimensions (int): Number of dimensions.
        domain (Tuple[float, float]): Domain of possible values.
        binary (bool): Whether to use binary representation.

    Returns:
        np.ndarray: Initialized population.
    """
    if binary:
        return np.random.randint(2, size=(pop_size, dimensions * 16))
    return np.random.uniform(domain[0], domain[1], (pop_size, dimensions))

In [6]:
def tournament_selection(pop: np.ndarray, scores: np.ndarray, k: int = 2) -> np.ndarray:
    """
    Perform tournament selection to choose an individual.

    Args:
        pop (np.ndarray): Population of individuals.
        scores (np.ndarray): Fitness scores of individuals.
        k (int): Tournament size.

    Returns:
        np.ndarray: Selected individual.
    """
    selected_indices = np.random.choice(len(pop), k, replace=False)
    best_index = selected_indices[np.argmin(scores[selected_indices])]
    return pop[best_index]

In [14]:
def crossover(parent1: np.ndarray, parent2: np.ndarray, binary: bool = False) -> Tuple[np.ndarray, np.ndarray]:
    """
    Perform crossover between two parents.

    Args:
        parent1 (np.ndarray): First parent.
        parent2 (np.ndarray): Second parent.
        binary (bool): Whether the representation is binary.

    Returns:
        Tuple[np.ndarray, np.ndarray]: Two offspring individuals.
    """
    if binary:
        # Perform uniform crossover
        mask = np.random.randint(0, 2, size=len(parent1)).astype(bool)
        child1 = np.where(mask, parent1, parent2)
        child2 = np.where(mask, parent2, parent1)
    else:
        # Arithmetic crossover
        alpha = np.random.rand()
        child1 = alpha * parent1 + (1 - alpha) * parent2
        child2 = alpha * parent2 + (1 - alpha) * parent1
    
    return child1, child2

In [12]:
def mutate(individual: np.ndarray, mutation_rate: float = 0.01, binary: bool = False, domain: Tuple[float, float] = (-5, 5)) -> np.ndarray:
    """
    Mutate an individual.

    Args:
        individual (np.ndarray): The individual to mutate.
        mutation_rate (float): Probability of mutation per gene.
        binary (bool): Whether the representation is binary.
        domain (Tuple[float, float]): Bounds for real-valued mutation.

    Returns:
        np.ndarray: Mutated individual.
    """
    if binary:
        mutation_mask = np.random.rand(len(individual)) < mutation_rate
        individual[mutation_mask] = 1 - individual[mutation_mask]
    else:
        individual += np.random.normal(0, 0.1, size=individual.shape)
        individual = np.clip(individual, domain[0], domain[1])
    return individual

In [8]:
def evolutionary_algorithm_with_best_series(pop_size: int, dimensions: int, domain: Tuple[float, float], max_evaluations: int,
                                             eval_func: Callable[[np.ndarray], float], binary: bool = False) -> List[float]:
    """
    Run an evolutionary algorithm and track the best series of fitness values.

    Args:
        pop_size (int): Population size.
        dimensions (int): Number of dimensions.
        domain (Tuple[float, float]): Bounds for the variables.
        max_evaluations (int): Maximum number of evaluations.
        eval_func (Callable[[np.ndarray], float]): Evaluation function.
        binary (bool): Whether to use binary representation.

    Returns:
        List[float]: Best fitness values tracked over evaluations.
    """
    pop = initialize_population(pop_size, dimensions, domain, binary)
    best_series = []
    eval_count = 0
    best_so_far = float('inf')

    while eval_count < max_evaluations:
        if binary:
            decoded_pop = binary_to_real(pop, domain, dimensions)
            scores = np.apply_along_axis(eval_func, 1, decoded_pop)
        else:
            scores = np.apply_along_axis(eval_func, 1, pop)

        for score in scores:
            best_so_far = min(best_so_far, score)
            best_series.append(best_so_far)
            eval_count += 1
            if eval_count >= max_evaluations:
                break

        # Generate new population
        new_pop = np.empty_like(pop)
        for i in range(0, pop_size, 2):
            parent1 = tournament_selection(pop, scores)
            parent2 = tournament_selection(pop, scores)
            child1, child2 = crossover(parent1, parent2, binary)
            new_pop[i] = mutate(child1, binary=binary, domain=domain)
            new_pop[i + 1] = mutate(child2, binary=binary, domain=domain)
        pop = new_pop

    return best_series

In [9]:
def run_experiments_with_best_series() -> Dict[str, List[List[float]]]:
    """
    Run multiple experiments with the evolutionary algorithm.

    Returns:
        Dict[str, List[List[float]]]: Results of the experiments grouped by settings.
    """
    results = {}
    results['F1_real'] = [evolutionary_algorithm_with_best_series(pop_size, dimensions, domain_f1, max_evaluations, f1, binary=False) for _ in range(100)]
    results['F2_real'] = [evolutionary_algorithm_with_best_series(pop_size, dimensions, domain_f2, max_evaluations, f2, binary=False) for _ in range(100)]
    results['F1_binary'] = [evolutionary_algorithm_with_best_series(pop_size, dimensions, domain_f1, max_evaluations, f1, binary=True) for _ in range(100)]
    results['F2_binary'] = [evolutionary_algorithm_with_best_series(pop_size, dimensions, domain_f2, max_evaluations, f2, binary=True) for _ in range(100)]
    return results

In [10]:
def save_best_series_to_excel(results: Dict[str, List[List[float]]]) -> None:
    """
    Save the results of the evolutionary algorithm to an Excel file.

    Args:
        results (Dict[str, List[List[float]]]): Results of the experiments.
    """
    with pd.ExcelWriter("results.xlsx", engine="openpyxl") as writer:
        for name, data in results.items():
            data = np.array(data)
            df_series = pd.DataFrame(data.T)
            df_series.to_excel(writer, sheet_name=f'{name}_best_series', index=False, header=False)

            # Generate mean plot
            mean_data = np.mean(data, axis=0)
            plt.figure()
            plt.plot(mean_data, label=f'Average Best Score ({name})')
            plt.xlabel("Evaluation")
            plt.ylabel("Fitness")
            plt.legend()
            plt.title(f'Average Best Scores - {name}')

            img_data = BytesIO()
            plt.savefig(img_data, format='png')
            img_data.seek(0)
            plt.close()

            # Add plot to Excel sheet
            workbook = writer.book
            worksheet = workbook[f'{name}_best_series']
            img = Image(img_data)
            worksheet.add_image(img, 'E5')


In [16]:
# Experiment variables
pop_size = 50
dimensions = 10
domain_f1 = (-3, 3)
domain_f2 = (-32.768, 32.768)
max_evaluations = 10000

In [12]:
# Run experiments and save results
results = run_experiments_with_best_series()
save_best_series_to_excel(results)